In [1]:
# import display libs
from IPython.display import Image
%matplotlib inline
from IPython.display import Latex

In [2]:
# import libs
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


# setup figure params
figureparams = {'axes.labelsize': 24,
           'axes.titlesize': 20,
           'axes.linewidth': 1.3,
           'font.size': 20,
           'legend.fontsize': 18,
           'figure.figsize': (10,7),
           'font.family': 'serif',
           'font.serif': 'Computer Modern Roman',
           'xtick.labelsize': 18,
           'xtick.major.size': 5.5,
           'xtick.major.width': 1.3,
           'ytick.labelsize': 18,
           'ytick.major.size': 5.5,
           'ytick.major.width': 1.3,
           'text.usetex': True,
           'figure.autolayout': True}
plt.rcParams.update(figureparams)
matplotlib.rcParams['text.usetex']=True
matplotlib.rcParams['text.latex.unicode']=True
matplotlib.get_configdir()

C:\Users\Edward\AppData\Roaming\Python\Python36\site-packages\matplotlib\__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


'C:\\Users\\Edward\\.matplotlib'

# Binomial test
### Contents
    1. Binomial test
        a. Right-sided test
        b. Left-sided test
        c. Two-sided test
    2. Confidence Intervals
        a. Clopper-Pearson
        b. Bootstrap

## 1. Binomial test

The binomial test is arguably the simplest of all statistical tests. Let's say we flip a coin $N$ times and we observe $k$ successes. Let's say we expected a probability $p \neq k/ N$. Of course, because of the variance of this process, we would not expect to observe exactly $pN$ successes. However, this begs the question: when can we say that our observed probability is statistically different from the expected probability?

We need to distinguish three cases. In all cases, our null hypothesis will be that the our expected probability is the real probability: 
- $H_0$: $p_\text{real} = p_\text{exp}$.

The difference is in the alternative hypotheses. We can have:
- $H_A$: $p_\text{real} \geq p_\text{exp}$;
- $H_A$: $p_\text{real} \leq  p_\text{exp}$;
- $H_A$: $p_\text{real} \neq p_\text{exp}$.

The first two are called one-sided tests, whereas the second is called a two-sided test. Often, the number of observations will hint at whether one wants to conduct a left- or right-sided test. The two-sided test is obviously the most general of the three. Let's now implement them.

#### 1a. Right-sided binomial test

To test the zero hypothesis, we would like to compute the probability that we observe $k$ or more successes in $N$ trials assuming the generating probability to be $p_\text{expected}$:
\begin{equation}
P\left( X \geq k \hspace{2pt} | \hspace{2pt} H_0   \right) = \sum_{i = k}^N {n \choose k} p_\text{exp}^k (1-p_\text{exp})^{n-k}.
\end{equation}

If this probability - often referred to as the p-value - is smaller than some preset significance level $\alpha$, we can reject the zero hypothesis in favor of the alternative hypothesis. Typical values for $\alpha$ are $0.05$ and $0.01$.


In [11]:
# set basic parameters
N = 1000        # number of obs
p = .32         # expected default probability

# generate data with underlying probability p_real
p_real = .40
rand_data = np.random.rand(N)
idx = np.where(rand_data < p_real)[0]
obs = np.zeros(N)                                 
obs[idx] = 1 
k = np.sum(obs).astype(int) # number of defaults

In [12]:
# One-sided Binomial test (greater)
def binom_test_greater(N, k, p):
    probs = []
    for i in range(k, N+1):
        fact = np.math.factorial(N) / ( np.math.factorial(i) * np.math.factorial(N-i) )
        tmp = fact * p**i * (1-p)**(N-i)
        probs.append(tmp)
    
    return np.sum(probs)
        
p_val = binom_test_greater(N, k, p)


# let's check with statsmodels
from statsmodels.stats.proportion import binom_test
p_val_scipy = binom_test(k, N, p, alternative='larger')

print("Our p-value: " + str(p_val))
print("Scipys p-value: " + str(p_val_scipy))

Our p-value: 6.702613548517755e-07
Scipys p-value: 6.702613548516461e-07


#### 1b. Left-sided binomial test

In this case, to test the zero hypothesis, we would like to compute the probability that we observe $k$ or less successes in $N$ trials assuming the generating probability to be $p_\text{expected}$:
\begin{equation}
P\left( X \leq k \hspace{2pt} | \hspace{2pt} H_0   \right) = \sum_{i = 0}^k {n \choose k} p_\text{exp}^k (1-p_\text{exp})^{n-k}.
\end{equation}



In [17]:
# set basic parameters
N = 1000        # number of obs
p = .32         # expected default probability

# generate data with underlying probability p_real
p_real = .20
rand_data = np.random.rand(N)
idx = np.where(rand_data < p_real)[0]
obs = np.zeros(N)                                 
obs[idx] = 1 
k = np.sum(obs).astype(int) # number of defaults

In [18]:
# One-sided Binomial test (smaller)
def binom_test_smaller(N, k, p):
    probs = []
    for i in range(0, k+1):
        fact = np.math.factorial(N) / ( np.math.factorial(i) * np.math.factorial(N-i) )
        tmp = fact * p**i * (1-p)**(N-i)
        probs.append(tmp)
    
    return np.sum(probs)
        
p_val = binom_test_smaller(N, k, p)


# let's check with statsmodels
from statsmodels.stats.proportion import binom_test
p_val_scipy = binom_test(k, N, p, alternative='smaller')

print("Our p-value: " + str(p_val))
print("Scipys p-value: " + str(p_val_scipy))

Our p-value: 2.056012793832713e-15
Scipys p-value: 2.0560127938321348e-15


#### 1c. Two-sided binomial test

The two-sided test tests whether at a given deviation $\delta = |k| - Np_\text{exp}$ the number of successes $k$ is too high or too low. The p-value is computed as
\begin{equation}
P\left( X \neq k \hspace{2pt} | \hspace{2pt} H_0   \right) = \sum_{i = 0}^{Np_\text{exp} - \delta} {n \choose k} p_\text{exp}^k (1-p_\text{exp})^{n-k} + \sum_{i = Np_\text{exp} + \delta}^{N} {n \choose k} p_\text{exp}^k (1-p_\text{exp})^{n-k}.
\end{equation}

Note that, for symmetrical distributions, the two-sided test will be equal to twice the left- or right-sided test.
However, the binomial distribution is not symmetrical.


In [19]:
# compute two-sided test: use same data as before
def binom_test_two_sided(N, k, p):
    probs = []
    dev = np.abs(k - N*p) 
    left_tail = np.round( N*p - dev ).astype(int)
    right_tail = np.round( N*p + dev ).astype(int)
    
    for i in range(0, left_tail+1):
        fact = np.math.factorial(N) / ( np.math.factorial(i) * np.math.factorial(N-i) )
        tmp = fact * p**i * (1-p)**(N-i)
        probs.append(tmp)        
    for i in range(right_tail, N+1):
        fact = np.math.factorial(N) / ( np.math.factorial(i) * np.math.factorial(N-i) )
        tmp = fact * p**i * (1-p)**(N-i)
        probs.append(tmp)           
    return np.sum(probs)
        
p_val = binom_test_two_sided(N, k, p)


# let's check with statsmodels
from statsmodels.stats.proportion import binom_test
p_val_scipy = binom_test(k, N, p, alternative='two-sided')

print("Our p-value: " + str(p_val))
print("Scipys p-value: " + str(p_val_scipy))

Our p-value: 8.327587638381893e-14
Scipys p-value: 3.5117235929402072e-15


## 2. Confidence intervals

Above, we used the p-value approach to say something about the statistical significance of a certain statistic (in this case the observed number of successes). Although often used (e.g. in science), some argue that it is better practice to report confidence intervals instead of significance levels. The reason would be that confidence intervals provide more information than significance levels. 

Often, there are two ways to go about setting up confidence intervals. The first is to find an analytical expression for the confidence interval. This is for example the case for the Clopper-Pearson confidence interval below. Another way is the bootstrap approach, in which one uses the bootstrap principle to recompute the test-statistic (in this case the number of successes) $N_\text{boots}$ times.

#### 2a. Clopper-Pearson

The Clopper-Pearson interval is given as $\left(\frac{k}{N} - \delta_1, \frac{k}{N} + \delta_2 \right)$ with confidence level $1- \alpha$, where $\delta_i$ is the largest value for which with significance $\alpha /2$ the following hypotheses tests succeed:
- $H_0$: $k_\text{real} = \frac{k}{n} - \delta_1$ and $H_A$: $k_\text{real} > \frac{k}{n} - \delta_1$;
- $H_0$: $k_\text{real} = \frac{k}{n} + \delta_2$ and $H_A$: $k_\text{real} < \frac{k}{n} - \delta_1$;

One can show that - because of the relation between the binomial distriubtion and the $\beta$-distribution - the CP-interval is equivalently given in terms of the $\beta$-distribution as
\begin{equation}
\left( B \left( \frac{\alpha}{2}, k, N-k-1 \right) , B \left( 1 - \frac{\alpha}{2}, k+1, N-k \right) \right),
\end{equation}
which allows for easy implementation.

In [10]:
from scipy.stats import beta

alpha = 0.05
CI_low = beta.ppf(alpha/2, k, N-k+1)
CI_up = beta.ppf(1-alpha/2, k+1, N-k)
print("with 95% conf., real p lies within: (" + str(np.round(CI_low, 2)) + ", " + str(np.round(CI_up,2)) + ")")
print("observed p: "+ str(np.round(k/N,2)))

with 95% conf., real p lies within: (0.17, 0.28)
observed p: 0.22


#### 2b. Bootstrap

Conceptually, it is easier to use bootstrapping. The idea is simple: given our sample, we compute our test statistic and then create a new sample by resampling from the sample with replacement. Then, we can recompute the test-statistic, which will be somewhat different. This process is the repeated $N_\text{boot}$ times. Finally, we can take the $\frac{\alpha}{2}$th and $1 - \frac{\alpha}{2}$th percentile of the resulting distribution of test statistics as confidence interval.

In [11]:
N = 1000
p = .32
rand_data = np.random.rand(N)
idx = np.where(rand_data < p)[0]
obs = np.zeros(N)                                 
obs[idx] = 1 

In [12]:
# let's check our expected probability
k = np.sum(obs)
p_expected = k / N

In [13]:
# let's use Clopper-Pearson to create a reference
alpha = 0.05
CI_low = beta.ppf(alpha/2, k, N-k+1)
CI_up = beta.ppf(1-alpha/2, k+1, N-k)
print("with 95% conf., real p lies within: (" + str(np.round(CI_low, 2)) + ", " + str(np.round(CI_up,2)) + ")")
print("observed p: "+ str(np.round(k/N,2)))  

with 95% conf., real p lies within: (0.32, 0.38)
observed p: 0.35


In [25]:
# let's now bootstrap the CI
def bootstrap(data, n=1000, func=np.sum):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(data)
    for i in range(n):
        itersample = np.random.choice(data, size=sample_size, replace=True)
        simulations.append(func(itersample) / sample_size)
    simulations.sort()
    return simulations

def confidence_interval(sim, alpha):
    """
    Return 2-sided symmetric confidence interval specified
    by p.
    """
    sim.sort()
    n = len(sim)
    u_pval = 1 - alpha/2
    l_pval = alpha / 2
    l_idx = int(np.floor(n*l_pval))
    u_idx = int(np.floor(n*u_pval))
    return(sim[l_idx], sim[u_idx])

In [26]:
# run the bootstrap
sim = bootstrap(obs)
bounds = confidence_interval(sim, alpha = 0.05)

print("Bootstrap: real p lies within: (" + str(bounds[0]) + ", " + str(bounds[1]) + ")")
print("Clopper-Pearson:, real p lies within: (" + str(np.round(CI_low, 2)) + ", " + str(np.round(CI_up,2)) + ")")
print("Observed p: "+ str(np.round(k/N,2)))  

Bootstrap: real p lies within: (0.319, 0.379)
Clopper-Pearson:, real p lies within: (0.32, 0.38)
Observed p: 0.35
